In [ ]:
import sys
import json
import glob
sys.path.append("..")

import pandas as pd
import numpy as np
import tqdm
import torch_geometric

from gvpgnn.datasets import ProteinGraphDataset, BatchSampler
from gvpgnn.paths import data_folder

In [ ]:
dataset_version = "cleaned_skip_missing"
split_name = "train"

dataset = ProteinGraphDataset(
  data_folder(f"{dataset_version}/{split_name}"),
  num_positional_embeddings=16,
  top_k=30,
  num_rbf=16,
  device="cpu"
)

print("Dataset size:", len(dataset))

# Make sure that we can iterate through the dataset:
# for i in range(min(len(dataset), 3)):
  # print(dataset[i])

In [ ]:
sampler = BatchSampler(
  dataset.node_counts,
  max_nodes=3000,
  sampler_weights=dataset.sampler_weights
)

dataloader = torch_geometric.loader.DataLoader(
  dataset,
  num_workers=1,
  batch_sampler=sampler,
)

N = 1
label_freq = np.zeros(10)

for i, batch in enumerate(dataloader):
  print(i)
  if i > N:
    break
  batch_labels = batch.task_label.numpy()
  for label in batch_labels:
    label_freq[label] += 1

In [ ]:
label_freq / label_freq.sum()